In [1]:
import numpy as np
from scipy.optimize import minimize

# Define the sigmoid and exponential functions
def sigmoid_func(x, L, x0, k, b):
    return L / (1 + np.exp(-k * (x - x0))) + b

def exponential_func_new(x, A, B, C):
    return A + B * np.exp(C * x)

# Given data and fitted models (from the case study and previous fits)
def iron_ore_feed_rate(x):
    return 0.02 * x**2 - 0.5 * x + 8.7

def dolomite_consumption(x):
    return -0.2 * x + 90

def campaign_days(x):
    return -0.01 * x**3 + 0.3 * x**2 - 2.5 * x + 62.5

def shutdown_days(x):
    return sigmoid_func(x, 11, 0.5, 10, 7)

def dolochar_generation(x):
    return 0.005 * x**3 - 0.1 * x**2 + 0.2 * x + 0.25

def steam_generation(x):
    return exponential_func_new(x, 9.5489, -0.9215, 1.19951)

def overhead_cost(x):
    return 100 * x + 1850

def dolochar_rate(x):
    return sigmoid_func(x, 500, 0.6, 15, 350)

# Constants
selling_price_AB_DRI = 30000  # INR/MT for A/B Grade DRI
selling_price_minus1mm_DRI = 18500  # INR/MT for -1 mm DRI
dolomite_cost_per_kg = 1.5  # INR/Kg

# Define the profit function
def compute_profit(x):
    iron_ore_feed = iron_ore_feed_rate(x)
    dolomite_cons = dolomite_consumption(x)
    campaign_days_count = campaign_days(x)
    shutdown_days_count = shutdown_days(x)
    dolochar_gen = dolochar_generation(x)
    steam_gen = steam_generation(x)
    overhead = overhead_cost(x)
    dolochar_rate_value = dolochar_rate(x)

    available_days = 365 - (365 / campaign_days_count) * shutdown_days_count
    production_per_day = iron_ore_feed * 24 * 0.555
    annual_production = production_per_day * available_days

    # Revenue from DRI (95% A/B Grade DRI, 5% -1 mm DRI)
    revenue_AB_DRI = (annual_production * 0.95) * selling_price_AB_DRI
    revenue_minus1mm_DRI = (annual_production * 0.05) * selling_price_minus1mm_DRI
    total_revenue_dri = revenue_AB_DRI + revenue_minus1mm_DRI

    # Dolochar revenue
    total_dolochar = annual_production * dolochar_gen
    revenue_dolochar = total_dolochar * dolochar_rate_value

    # Steam revenue
    total_steam = steam_gen * 24 * available_days
    revenue_steam = total_steam * 500

    # Cost calculations
    iron_ore_consumed = iron_ore_feed * 24 * available_days
    cost_iron_ore = iron_ore_consumed * 8400

    # Coal cost
    avg_cfe_ratio = x * 0.402 + (1 - x) * 0.430
    avg_fc_coal = x * 0.52 + (1 - x) * 0.42
    avg_moisture_coal = x * 0.09 + (1 - x) * 0.08
    mc_coal = 1 - avg_moisture_coal
    average_feed_rate_coal = (iron_ore_feed * avg_cfe_ratio * 0.64 * 0.97) / (avg_fc_coal * mc_coal)
    coal_consumed = average_feed_rate_coal * 24 * available_days
    cost_coal = coal_consumed * (x * 9800 + (1 - x) * 5000)

    # Dolomite cost
    total_dolomite_consumed = (dolomite_cons / 1000) * annual_production
    cost_dolomite = total_dolomite_consumed * dolomite_cost_per_kg * 1000

    # Overhead cost
    total_overhead_cost = overhead * available_days

    # Total profit
    total_profit = (total_revenue_dri + revenue_dolochar + revenue_steam) - (cost_iron_ore + cost_coal + cost_dolomite + total_overhead_cost)

    return -total_profit  # Minimizing negative profit to maximize profit

# Set the bounds for the optimization (fraction of imported coal)
bounds = [(0.3, 1.0)]

# Perform the optimization using scipy's minimize function
result = minimize(compute_profit, x0=0.5, bounds=bounds, method='SLSQP')

# Optimal fraction of imported coal and maximum profit
optimal_fraction = result.x[0]
max_profit = -result.fun  # Since we minimized negative profit

print(f"Optimal Fraction of Imported Coal: {optimal_fraction:.4f}")
print(f"Maximum Profit: {max_profit:.2f} INR/year")


Optimal Fraction of Imported Coal: 0.5000
Maximum Profit: 232807177.19 INR/year
